# ROC Curves 
Normal, Laplace, and Cauchy with Shifted Means and Variances

## Import packages and define functions.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import log
from scipy.stats import laplace, cauchy, norm

plt.close("all")

def log_likelihood_ratio(pdfs):
    """
    Log likelihood ratio of two pdfs.
    """
    return log(pdfs[1])-log(pdfs[0])
    
def likelihood_ratio(pdfs):
    """
    Likelihood ratio of two pdfs.
    """
    return pdfs[1]/pdfs[0]

def positive_regions(thresh,x,likelihood):
    """
    Find the regions of the domain in which the likelihood is greater than the 
    given threshold. 
    """
    regions = []    
    left=None
    last_i = likelihood.shape[0]-1
    for i,val in enumerate(likelihood):
        if left is None:
            if val >= thresh:
                left = x[i]
        else:
            if val <= thresh:
                regions.append((left,x[i-1]))
                left=None
            elif i == last_i:
                regions.append((left,x[i]))
    return regions
            
def probability_of_regions(regions,dist_fun,pdf_parameters):
    """
    Calculates the total probability of one or more regions in the domain 
    of the given pdf. 
    """
    prob = 0.
    for r in regions:
        prob += (dist_fun.cdf(r[1],**pdf_parameters) - 
                    dist_fun.cdf(r[0],**pdf_parameters) )
    return prob

def ROC_from_pdf(x,like_fun,dist_fun, pdf_parameters,name):
    """
    Plots the the Negative and Positive probability density functions, the 
    likelihood, and ROC.
    
    PARAMETERS
    ----------
    x: ndarray
        Domain over which to evalulate pdfs.
    like_fun: function
        Function that calculates likelihood.
    dist_fun: function
        One of the scipy.stats distribution functions.
    pdf_parameters: tuple 
        `pdf_parameters` is a tuple of length 2 that contains the  
        parameters of the negative and positive pdf parameters, respectively.
    name: str
        What to call the resulting figure.
    """
    # setup figure
    fig, plots = plt.subplots(1,2,figsize=(12,6))
    fig.canvas.set_window_title(name)
    pdf_plot, roc_plot = plots
    pdf_plot.set_title("PDF")
    pdf_plot.set_xlabel("x")
    pdf_plot.set_ylabel("p(x)")
    roc_plot.set_title("ROC")
    roc_plot.set_xlabel("Probability False Alarm")
    roc_plot.set_ylabel("Probability Detection")
    threshold_plot = pdf_plot.twinx()
    if like_fun == likelihood_ratio:
        threshold_plot.set_ylabel("Likelihood Ratio")
    elif like_fun == log_likelihood_ratio:
        threshold_plot.set_ylabel("Log Likelihood Ratio")
    fig.tight_layout()
    
    # find negative and positive pdfs
    pdfs = (dist_fun.pdf(x,**pdf_parameters[0]), dist_fun.pdf(x,**pdf_parameters[1]) )
    
    # plot pdfs
    pdf_plot.plot(x,pdfs[0],'b-',label="Negative")
    pdf_plot.plot(x,pdfs[1],'r-',label="Positive")
    pdf_plot.legend()
    
    # find and plot likelihood
    like = like_fun(pdfs)
    threshold_plot.plot(x,like,'k--')
    
    # calculate probability of false alarm and false positive
    P_detect = np.zeros_like(x)
    P_false_alarm = np.zeros_like(x)
    for i,thresh in enumerate(like):
        regions = positive_regions(thresh,x,like)
        P_detect[i]  = probability_of_regions(regions,dist_fun,pdf_parameters[1])
        P_false_alarm[i]  = probability_of_regions(regions,dist_fun,pdf_parameters[0])

    # plot ROC
    roc_plot.plot(P_false_alarm,P_detect,'g')

## Define domain and PDF parameters.

In [2]:
# domain over which to evalulate pdfs
x = np.arange(-20, 20, 0.01)

# negative pdf parameters
params_negative = dict(loc=-2,scale=1)
# positive pdf parameters
params_positive = dict(loc=2,scale=2)
# package parameters
pdf_params = (params_negative,params_positive)

## Normal

In [3]:
# draw ROC for normal distribution 
ROC_from_pdf(x,log_likelihood_ratio,norm,pdf_params,"Normal")

plt.show()

## Cauchy

In [4]:
# draw ROC for cauchy distribution 
ROC_from_pdf(x,likelihood_ratio,cauchy,pdf_params,"Cauchy")

plt.show()

## Laplace

In [5]:
# draw ROC for laplace distribution 
ROC_from_pdf(x,log_likelihood_ratio,laplace,pdf_params,"Laplace")

plt.show()